## Exploring metadata of problematic series (registration-wise)

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, clear_output
import itk
import SimpleITK as sitk
import matplotlib.patches as patches
from ipywidgets import interact, fixed
from matplotlib.widgets import Slider
from Nyul_preprocessing import *

In [ ]:
def display_images(fixed_image_z, moving_image_z, fixed_npa, moving_npa):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
    
    # Draw the fixed image in the first subplot.
    plt.subplot(1,2,1)
    plt.imshow(fixed_npa[fixed_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('fixed image')
    plt.axis('off')
    
    # Draw the moving image in the second subplot.
    plt.subplot(1,2,2)
    plt.imshow(moving_npa[moving_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('moving image')
    plt.axis('off')
    
    plt.show()

# overlaps images and allows to change alpha blending -> opacity
def display_images_with_alpha(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

In [ ]:
fixed_image = sitk.ReadImage('/local_ssd/practical_wise24/prostate_cancer/clean_dataset/BREST_011/Pelvis_ep2d_diff_tra.mha', sitk.sitkFloat32) 
moving_image =  sitk.ReadImage('/local_ssd/practical_wise24/prostate_cancer/PICAIDataset/input/images/11443/11443_1001467_t2w.mha', sitk.sitkFloat32)

In [ ]:
interact(display_images, fixed_image_z=(0,fixed_image.GetSize()[2]-1), moving_image_z=(0,moving_image.GetSize()[2]-1), fixed_npa = fixed(sitk.GetArrayViewFromImage(fixed_image)), moving_npa=fixed(sitk.GetArrayViewFromImage(moving_image)));

#### Fixed image info

In [ ]:
print(fixed_image.GetOrigin())
print(fixed_image.GetSize())
print(fixed_image.GetSpacing())
print(fixed_image.GetDirection())

#### Moving image info

In [ ]:
print(moving_image.GetOrigin())
print(moving_image.GetSize())
print(moving_image.GetSpacing())
print(moving_image.GetDirection())

#### Example registration

In [ ]:
# Set up the registration method
registration_method = sitk.ImageRegistrationMethod()

# Similarity metric settings
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

# Interpolator
registration_method.SetInterpolator(sitk.sitkHammingWindowedSinc)

# Optimizer settings
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
registration_method.SetOptimizerScalesFromPhysicalShift()

# Setup for the multi-resolution framework
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# Initial alignment of the moving image
initial_transform = sitk.CenteredTransformInitializer(res, adc_image, sitk.Euler3DTransform(), sitk.CenteredTransformInitializerFilter.GEOMETRY)
registration_method.SetInitialTransform(initial_transform, inPlace=False)

# Execute the registration
final_transform = registration_method.Execute(res, adc_image)



In [ ]:
# Resample the moving image to the space of the fixed image
# resampler1 = sitk.ResampleImageFilter()
# resampler1.SetReferenceImage(fixed_image)
# resampler1.SetInterpolator(sitk.sitkBSpline)
# resampler1.SetTransform(final_transform)
# aligned_moving_image1 = resampler1.Execute(moving_image)

# resampler2 = sitk.ResampleImageFilter()
# resampler2.SetReferenceImage(fixed_image)
# resampler2.SetInterpolator(sitk.sitkLinear)
# resampler2.SetTransform(final_transform)
# aligned_moving_image2 = resampler2.Execute(moving_image)

resampler4 = sitk.ResampleImageFilter()
resampler4.SetReferenceImage(res)
resampler4.SetInterpolator(sitk.sitkHammingWindowedSinc)
resampler4.SetTransform(final_transform)
resampled = resampler4.Execute(adc_image)

In [ ]:
interact(display_images, fixed_image_z=(0,fixed_image.GetSize()[2]-1), moving_image_z=(0,aligned_moving_image1.GetSize()[2]-1), fixed_npa = fixed(sitk.GetArrayViewFromImage(fixed_image)), moving_npa=fixed(sitk.GetArrayViewFromImage(aligned_moving_image1)));
interact(display_images, fixed_image_z=(0,fixed_image.GetSize()[2]-1), moving_image_z=(0,aligned_moving_image2.GetSize()[2]-1), fixed_npa = fixed(sitk.GetArrayViewFromImage(fixed_image)), moving_npa=fixed(sitk.GetArrayViewFromImage(aligned_moving_image2)));
interact(display_images, fixed_image_z=(0,fixed_image.GetSize()[2]-1), moving_image_z=(0,aligned_moving_image4.GetSize()[2]-1), fixed_npa = fixed(sitk.GetArrayViewFromImage(fixed_image)), moving_npa=fixed(sitk.GetArrayViewFromImage(aligned_moving_image4)));

In [ ]:

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2]-1), alpha=0.5, fixed = fixed(fixed_image), moving=fixed(moving_image));

#### Visualize overlapped slices

In [ ]:
fixed_image = sitk.ReadImage('/local_ssd/practical_wise24/prostate_cancer/registered_picai_dataset/10005/10005_1000005_t2w.mha', sitk.sitkFloat32) 
moving_image =  sitk.ReadImage('/local_ssd/practical_wise24/prostate_cancer/registered_picai_dataset/10005/10005_1000005_adc.mha', sitk.sitkFloat32)

In [ ]:
# Convert SimpleITK images to numpy arrays
np_image1 = sitk.GetArrayFromImage(fixed_image)
np_image2 = sitk.GetArrayFromImage(moving_image)

# Function to display a slice
def display_slice(slice_number):
    plt.imshow(np_image1[slice_number, :, :], cmap='gray')
    plt.imshow(np_image2[slice_number, :, :], cmap='hot', alpha=0.6)
    plt.axis('off')
    plt.show()

# Initial slice number
initial_slice = 48

# Create a figure and axis for the plot
fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)

# Display the initial slice
display_slice(initial_slice)

plt.show()

In [ ]:
def apply_centered_transform(fixed_image, moving_image):
    initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                        moving_image, 
                                                        sitk.Euler3DTransform(), 
                                                        sitk.CenteredTransformInitializerFilter.GEOMETRY)

    reference_image = sitk.Image(fixed_image.GetSize()[0], fixed_image.GetSize()[1], fixed_image.GetSize()[2], fixed_image.GetPixelID())
    # out_size = [
    #     int(np.round(
    #         size * (spacing_in / spacing_out)
    #     ))
    #     for size, spacing_in, spacing_out in zip([384,384,22], [0.5, 0.5, 3.000000075935057], [0.71875,0.71875, 1.0])
    # ]
    # reference_image = sitk.Image(out_size[0], out_size[1], out_size[2], fixed_image.GetPixelID())
    reference_image.SetOrigin(fixed_image.GetOrigin())
    reference_image.SetSpacing(fixed_image.GetSpacing())
    reference_image.SetDirection(moving_image.GetDirection())
    # initial_transform =sitk.Transform()
    return sitk.Resample(moving_image, reference_image, initial_transform, sitk.sitkHammingWindowedSinc, 0.0, moving_image.GetPixelID())

In [ ]:
resampled = apply_centered_transform(fixed_image=fixed_image, moving_image=moving_image)

In [ ]:

interact(display_images, fixed_image_z=(0,fixed_image.GetSize()[2]-1), moving_image_z=(0,resampled.GetSize()[2]-1), fixed_npa = fixed(sitk.GetArrayViewFromImage(fixed_image)), moving_npa=fixed(sitk.GetArrayViewFromImage(resampled)));

In [ ]:
print(resampled.GetOrigin())
print(resampled.GetSize())
print(resampled.GetSpacing())
print(resampled.GetDirection())

In [ ]:
sitk.WriteImage(resampled, './test_adc.mha')

In [ ]:
private_image = sitk.ReadImage('/local_ssd/practical_wise24/prostate_cancer/mha_raw_images/BREST_001/Pelvis_ep2d_diff_tra_ADC.mha', sitk.sitkFloat32) 
picai_image =  sitk.ReadImage('/local_ssd/practical_wise24/prostate_cancer/PICAIDataset/input/images/10012/10012_1000012_adc.mha', sitk.sitkFloat32)

In [ ]:
image1_array = sitk.GetArrayFromImage(private_image)
image2_array = sitk.GetArrayFromImage(picai_image)
# Plot histograms
plt.figure(figsize=(10, 5))
plt.hist(image1_array.flatten(), bins=50, alpha=0.5, label='Image 1')
plt.hist(image2_array.flatten(), bins=50, alpha=0.5, label='Image 2')
plt.xlabel('Intensity')
plt.ylabel('Frequency')
plt.title('Intensity Distributions')
plt.legend()
plt.show()

# Calculate and print statistical measures
print("Image 1: Mean =", np.mean(image1_array), ", Std Dev =", np.std(image1_array))
print("Image 2: Mean =", np.mean(image2_array), ", Std Dev =", np.std(image2_array))

In [ ]:
model_path = './nyul_trained_models/Pelvis_ep2d_diff_tra_ADC.npz'
f = np.load(model_path, allow_pickle=True)
Model = f['trainedModel'].all()
meanLandmarks = Model['meanLandmarks']
standarized_image = transform(picai_image,meanLandmarks=meanLandmarks)

In [ ]:
image1_array = sitk.GetArrayFromImage(private_image)
image2_array = sitk.GetArrayFromImage(standarized_image)
# Plot histograms
plt.figure(figsize=(10, 5))
plt.hist(image1_array.flatten(), bins=50, alpha=0.5, label='Image 1')
plt.hist(image2_array.flatten(), bins=50, alpha=0.5, label='Image 2')
plt.xlabel('Intensity')
plt.ylabel('Frequency')
plt.title('Intensity Distributions')
plt.legend()
plt.show()

# Calculate and print statistical measures
print("Image 1: Mean =", np.mean(image1_array), ", Std Dev =", np.std(image1_array))
print("Image 2: Mean =", np.mean(image2_array), ", Std Dev =", np.std(image2_array))

In [ ]:
root_dir = '/local_ssd/practical_wise24/prostate_cancer/PICAIDataset/input/images'
counter1  = 0
counter2 = 0
counter3 = 0
found = False
for patient_folder in os.listdir(root_dir):
    files_path = os.path.join(root_dir, patient_folder)
    for mha_file_name in os.listdir(files_path):
        description = mha_file_name.split('.')[0]
        if 't2w' in description:
            image = sitk.ReadImage(os.path.join(files_path, mha_file_name), sitk.sitkFloat32)
            if image.GetSize()[0] == 1024:
                counter1 += 1
                found = True
            elif image.GetSize()[0] == 640:
                counter2 +=1
            elif image.GetSize()[0] == 384:
                counter3 +=1
        if found:
            if 'adc' in description:
                adc_image = sitk.ReadImage(os.path.join(files_path, mha_file_name), sitk.sitkFloat32)
                break
    if found:
        break

In [ ]:
image.GetSize()

In [ ]:
def crop_sitk_image_centered(sitk_image, offset, size):
    # Get the size of the image
    image_size = sitk_image.GetSize()

    # Assume 3D image; for 2D, ignore the z-dimension
    image_center = [image_size[i] // 2 for i in range(len(image_size))]

    # Adjust the center by the offset
    adjusted_center = [image_center[i] + offset[i] for i in range(len(offset))]

    # Calculate the top-left corner of the crop region
    start_point = [adjusted_center[i] - size[i] // 2 for i in range(len(size))]

     # Using RegionOfInterest to crop the image
    roi_filter = sitk.RegionOfInterestImageFilter()
    roi_filter.SetSize(size)
    roi_filter.SetIndex(start_point)
    cropped_sitk_image = roi_filter.Execute(sitk_image)
    return cropped_sitk_image

In [ ]:
image = crop_sitk_image_centered(image, [0,0,0], [612,612,image.GetSize()[2]])

In [ ]:
image.GetSize()

In [ ]:
out_size = [
    int(np.round(
        size * (spacing_in / spacing_out)
    ))
    for size, spacing_in, spacing_out in zip([image.GetSize()[0],image.GetSize()[1],image.GetSize()[2]], [image.GetSpacing()[0],image.GetSpacing()[1],image.GetSpacing()[2]], [0.71875,0.71875, 1.0])
]
reference_image = sitk.Image(out_size[0], out_size[1], out_size[2], image.GetPixelID())
reference_image.SetOrigin(image.GetOrigin())
reference_image.SetSpacing([0.71875,0.71875, 1.0])
reference_image.SetDirection(image.GetDirection())
initial_transform =sitk.Transform()
res = sitk.Resample(image, reference_image, initial_transform, sitk.sitkHammingWindowedSinc, 0.0, image.GetPixelID())

In [ ]:
res.GetSize()

In [ ]:
interact(display_images, fixed_image_z=(0,image.GetSize()[2]-1), moving_image_z=(0,res.GetSize()[2]-1), fixed_npa = fixed(sitk.GetArrayViewFromImage(image)), moving_npa=fixed(sitk.GetArrayViewFromImage(res)));